# Runkeeper munging

https://runkeeper.com/exportData

* make an export
* download the ZIP
* unzip it into `./export/`

In [1]:
from lxml.etree import parse,dump
from pathlib import Path
from osgeo import ogr
import datetime
import geopandas as gpd
import folium
from shapely.geometry import Polygon, LineString
import json

p = Path('export')
walks = list(p.glob('*.gpx'))

In [2]:
def parse_gpx(p):
    doc = parse(p.open())
    name = doc.xpath('gpx:trk/gpx:name',namespaces={'gpx': 'http://www.topografix.com/GPX/1/1'})[0].text
    time = doc.xpath('gpx:trk/gpx:time',namespaces={'gpx': 'http://www.topografix.com/GPX/1/1'})[0].text
    route = [(float(trkpt.attrib['lon']),float(trkpt.attrib['lat'])) 
             for trkpt in doc.xpath('//gpx:trkpt',namespaces={'gpx': 'http://www.topografix.com/GPX/1/1'})]
    d = {
        'name': name,
        'time': datetime.datetime.strptime(time, "%Y-%m-%dT%H:%M:%SZ"),
        'geometry': LineString(route)
    }
    
    return d


In [3]:
%matplotlib inline
routes = [parse_gpx(walk) for walk in walks]

In [4]:
df = gpd.GeoDataFrame(routes)
df.sort_values('time',inplace=True)


In [12]:
def style_function(feature):
    return {
        'weight': 2,
        'opacity': .5,
        'color': 'black' if feature['properties']['type'] == 'Cycling' else 'white'
    }

home = [42.045465,-87.7161625]
m = folium.Map(
    location=[42.03,-87.7161625],#home,
    tiles='OpenStreetMap',
    zoom_start=15,
    height=2500
)

j = json.loads(df['geometry'].to_json())
for i,feature in enumerate(j['features']):
    row = df.loc[i].to_dict()
    feature['properties']['name'] = row['name']
    feature['properties']['type'] = row['name'].split(' ')[0]
    feature['properties']['date'] = row['time'].strftime("%Y-%m-%d"),

folium.GeoJson(
    j,
    name='geojson',
    style_function=style_function
).add_to(m)

m

In [6]:
len(j['features'])

76

In [7]:
j['features'][0]['properties']

{'date': ('2020-04-01',), 'name': 'Walking 4/1/20 5:36 pm', 'type': 'Walking'}

In [8]:
j2 = json.load(open("paths.geojson"))

In [9]:
j2['features'][0]['properties']

{'date': '2020-03-13', 'name': 'Walking 3/13/20 5:29 pm', 'type': 'Walking'}

In [10]:
for f in j2['features']:
    print(f['properties']['type'])

Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Cycling
Walking
Cycling
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Cycling
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Walking
Cycling
Cycling
